#### Task: get the transformer to respond to "What is love?" and "Love is what?" Answer: "Awesome"

In [2]:
# imports

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

In [4]:
# data

token_to_id = {
    'what': 0,
    'is': 1,
    'love': 2,
    'awesome': 3,
    '<EOS>': 4,
}

id_to_token = dict(map(reversed, token_to_id.items()))